In [1]:
# Verify we're in the correct working directory
import os
os.getcwd()   

'/Users/seraphinashi/Desktop/l3c_ctml'

In [2]:
# pip install -r requirements.txt

In [3]:
from src.STEP1_feature import a_cohort as a
from src.STEP1_feature import b_medication as b
from src.STEP1_feature import c_diagnosis as c
from src.STEP1_feature import d_lab_measures as d
from src.STEP1_feature import e_comorbidity as e
from src.STEP1_feature import i_combine_synthetic_data as i
import os
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

In [4]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("l3c_ctml") \
        .getOrCreate()

# Read CSV file into table
data_path = "synthetic_data/training"

schema = StructType([
    StructField('table', StringType(), True),
    StructField('field', StringType(), True),
    StructField('type', StringType(), True)
])

all_fields = spark.createDataFrame([], schema)
    
for path, subdirs, files in os.walk(data_path):
    for name in files:
        if "csv" in name and name[0]!=".":
            # table_fields = [(table_name, f.name, str(f.dataType)) for f in datafile.schema.fields]
            full_file = (os.path.join(path, name))
            df = spark.read.csv(os.path.join(path, name), header=True, inferSchema=True)
            var_name = name[:-4]
            print(full_file)
            globals()[var_name] = df

synthetic_data/training/observation_period.csv
synthetic_data/training/drug_exposure.csv
synthetic_data/training/death.csv
synthetic_data/training/concept.csv
synthetic_data/training/measurement.csv
synthetic_data/training/condition_occurrence.csv
synthetic_data/training/microvisits_to_macrovisits.csv
synthetic_data/training/visit_occurrence.csv
synthetic_data/training/long_covid_silver_standard.csv
synthetic_data/training/person.csv
synthetic_data/training/observation.csv
synthetic_data/training/procedure_occurrence.csv


In [5]:
# 1_cohort_and_features

covid_pasc_index_dates = a.sql_statement_01(long_covid_silver_standard)
cohort = a.sql_statement_00(covid_pasc_index_dates, person)
long_covid_patients = a.sql_statement_08(covid_pasc_index_dates)

hosp_cases = a.sql_statement_04(cohort, condition_occurrence, microvisits_to_macrovisits)

hosp_and_non = a.sql_statement_06(cohort, hosp_cases)

Feature_Table_Builder_v0 = a.sql_statement_03(covid_pasc_index_dates, hosp_and_non, microvisits_to_macrovisits)
icu_visits = a.sql_statement_05(microvisits_to_macrovisits, concept) # empty for some reason
inpatient_visits = a.sql_statement_07(microvisits_to_macrovisits, concept)

tot_icu_days_calc = a.sql_statement_09(Feature_Table_Builder_v0, icu_visits)
tot_ip_days_calc = a.sql_statement_10(Feature_Table_Builder_v0, inpatient_visits)

Feature_Table_Builder = a.sql_statement_02(Feature_Table_Builder_v0, tot_icu_days_calc, tot_ip_days_calc)

In [6]:
# 2_med_feature_table
DrugConcepts = b.sql_statement_00(concept)
Drugs_for_These_Patients = b.sql_statement_01(Feature_Table_Builder, drug_exposure)

drugRollUp = b.sql_statement_04(DrugConcepts, Drugs_for_These_Patients)

covid_drugs = b.sql_statement_02(Feature_Table_Builder, drugRollUp)
pre_pre_drugs = b.sql_statement_10(Feature_Table_Builder, drugRollUp)
pre_drugs = b.sql_statement_07(Feature_Table_Builder, drugRollUp)
post_drugs = b.sql_statement_05(Feature_Table_Builder, drugRollUp)

covidtbl = b.sql_statement_03(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
prepretbl = b.sql_statement_11(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
pretbl = b.sql_statement_12(Feature_Table_Builder, pre_pre_drugs, microvisits_to_macrovisits)
posttbl = b.sql_statement_06(Feature_Table_Builder, post_drugs, microvisits_to_macrovisits)

pre_post_med_count = b.sql_statement_08(covidtbl, posttbl, prepretbl, pretbl)

pre_post_med_count_clean = b.sql_statement_09(Feature_Table_Builder, pre_post_med_count)

In [7]:
pre_post_med_count_clean

DataFrame[person_id: double, patient_group: string, ancestor_drug_concept_name: string, ancestor_drug_concept_id: int, pre_pre_med_count: bigint, pre_med_count: bigint, covid_med_count: bigint, post_med_count: bigint, apprx_age: int, sex: int, race: int, ethn: int, tot_long_data_days: bigint, op_post_visit_ratio: double, post_ip_visit_ratio: double, covid_ip_visit_ratio: double, post_icu_visit_ratio: double, covid_icu_visit_ratio: double]

In [8]:
# 3_dx_feature_table
pre_pre_condition = c.sql_statement_05(Feature_Table_Builder, condition_occurrence)
pre_condition = c.sql_statement_03(Feature_Table_Builder, condition_occurrence)
covid_condition = c.sql_statement_00(Feature_Table_Builder, condition_occurrence)
post_condition = c.sql_statement_02(Feature_Table_Builder, condition_occurrence)

four_windows_dx_counts = c.sql_statement_01(Feature_Table_Builder, microvisits_to_macrovisits, pre_pre_condition, pre_condition, covid_condition, post_condition)

pre_post_dx_count_clean = c.sql_statement_04(Feature_Table_Builder, four_windows_dx_counts)


In [9]:
# 4_lab_measure_table
measure_person = d.measurement_person(measurement, Feature_Table_Builder, concept)
pre_pre_measurement = d.sql_statement_05(Feature_Table_Builder, measure_person )

pre_measurement = d.sql_statement_04( Feature_Table_Builder, measure_person )
covid_measurement = d.sql_statement_00(Feature_Table_Builder, measure_person )
post_measurement = d.sql_statement_03(Feature_Table_Builder, measure_person )

four_windows_measure = d.sql_statement_01(covid_measurement, post_measurement, pre_measurement, pre_pre_measurement)

lab_measures_clean = d.sql_statement_02(four_windows_measure)

# 5_comorbidity_table
high_level_condition_occur = e.sql_statement_01(Feature_Table_Builder, condition_occurrence, concept)

comorbidity_counts = e.sql_statement_00(Feature_Table_Builder, high_level_condition_occur)

In [10]:
# 6_covid_measures

In [11]:
# 9_obs

In [12]:
# i_combine

In [13]:
pre_post_med_final_distinct = i.pre_post_med_final_distinct(pre_post_med_count_clean)

#--------------------------
condition_rollup = i.condition_rollup(long_covid_patients, pre_post_dx_count_clean, concept)

parent_condition_rollup = i.parent_condition_rollup(condition_rollup)

final_rollups = i.final_rollups(condition_rollup, parent_condition_rollup)
pre_post_more_in_dx_calc = i.pre_post_more_in_dx_calc(pre_post_dx_count_clean)

add_alt_rollup = i.add_alt_rollup(final_rollups, pre_post_more_in_dx_calc)

pre_post_dx_final = i.pre_post_dx_final(add_alt_rollup)
#--------------------------

In [14]:
import pandas as pd
import re

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.context import SparkContext
ctx = SparkContext.getOrCreate()

In [17]:
def add_labels(pre_post_dx_count_clean, pre_post_med_count_clean, long_covid_patients, Feature_table_builder):

    target_columns = ['person_id', 'sex', 'patient_group', 'apprx_age', 'race', 'ethn', 'tot_long_data_days', 
        'op_post_visit_ratio', 'post_ip_visit_ratio', "covid_ip_visit_ratio", "post_icu_visit_ratio", "covid_icu_visit_ratio", 'min_covid_dt']

    df = Feature_Table_Builder.withColumnRenamed('min_covid_dt_2', 'min_covid_dt')
    df_unique_cohort_rows = df.select(target_columns).distinct()

    df =  df_unique_cohort_rows   

    df = df.toPandas()
    # df.isna().sum() # no missing values 
    
    # encode these caterogical variables as binaries
    df = pd.get_dummies(df, columns=["sex","race","ethn"])
    df = df.rename(columns = lambda c: str.lower(c.replace(" ", "_")))
    df = spark.createDataFrame(df) # convert pandas back to spark

    # Add Labels
    final_cols = df.columns
    final_cols.extend(["long_covid", "hospitalized", 'date_encode', 'season_covid'])
    final_cols.remove('min_covid_dt')
    
    df = df.join(long_covid_patients, on='person_id', how='inner')
    
    # Join with the long covid clinic data to build our labels (long_covid)
    df = df.withColumn("long_covid", F.when(df["pasc_index"].isNotNull(), 1).otherwise(0))
    df = df.withColumn("hospitalized", F.when(df["patient_group"] == 'CASE_HOSP', 1).otherwise(0))

    # add month, year based on covid_index
    # earliest_dt = min(df['min_covid_dt'])
    earliest_dt = df.agg({"min_covid_dt": "min"}).collect()[0][0]
    df = df.withColumn('date_encode', F.months_between(df['min_covid_dt'], F.lit(earliest_dt), False).cast(IntegerType()))
    df = df.withColumn('month', F.month(df['min_covid_dt']))
    df = df.withColumn('season_covid', F.when((df['month']>=3) & (df['month']<=5), 1).\
                                           when((df['month']>=6) & (df['month']<=8), 2).\
                                           when((df['month']>=9) & (df['month']<=11),3).\
                                           otherwise(4))

    df = df.select(final_cols)
    return df

In [18]:
add_labels = add_labels(pre_post_dx_count_clean, pre_post_med_count_clean, long_covid_patients, Feature_Table_Builder)

Py4JJavaError: An error occurred while calling o2003.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 67.0 failed 1 times, most recent failure: Lost task 4.0 in stage 67.0 (TID 5962) (10.0.0.80 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/seraphinashi/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.9 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spa